In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Autokeras

Link: https://autokeras.com/

**Installs**

In [ ]:
pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 29.6 MB/s 
     |████████████████████████████████| 133 kB 78.9 MB/s 


**Imports**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import autokeras as ak
import os
from sklearn.datasets import load_files

# **Image Classification**

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)


11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
[5 0 4]


In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=2)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))


Trial 1 Complete [00h 04m 30s]
val_loss: 0.051944199949502945

Best val_loss So Far: 0.051944199949502945
Total elapsed time: 00h 04m 30s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
1875/1875 [==============================] - 142s 76ms/step - loss: 0.1591 - accuracy: 0.9510
Epoch 2/2
1875/1875 [==============================] - 143s 76ms/step - loss: 0.0724 - accuracy: 0.9777
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets
313/313 [==============================] - 6s 19ms/step
[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]
313/313 [==============================] - 6s 19ms/step - loss: 0.0571 - accuracy: 0.9825
[0.057086821645498276, 0.9825000166893005]


# **Text classification**

In [ ]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)

# set path to dataset
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

classes = ["pos", "neg"]
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories=classes
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories=classes
)

x_train = np.array(train_data.data)
y_train = np.array(train_data.target)
x_test = np.array(test_data.data)
y_test = np.array(test_data.target)

print(x_train.shape)  # (25000,)
print(y_train.shape)  # (25000, 1)
print(x_train[0][:50])  # this film was just brilliant casting


84140032/84125825 [==============================] - 6s 0us/step
(25000,)
(25000,)
b'Zero Day leads you to think, even re-think why two'


In [ ]:
# Initialize the text classifier.
clf = ak.TextClassifier(
    overwrite=True, max_trials=1
)  # It only tries 1 model as a quick demo.
# Feed the text classifier with training data.
clf.fit(x_train, y_train, epochs=2)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
score = clf.evaluate(x_test, y_test)
print(score)


Trial 1 Complete [00h 04m 09s]
val_loss: 0.27494117617607117

Best val_loss So Far: 0.27494117617607117
Total elapsed time: 00h 04m 09s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
782/782 [==============================] - 144s 183ms/step - loss: 0.4390 - accuracy: 0.7727
Epoch 2/2
782/782 [==============================] - 145s 185ms/step - loss: 0.2400 - accuracy: 0.9065
INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets
782/782 [==============================] - 38s 49ms/step - loss: 0.2643 - accuracy: 0.8936
[0.2642713189125061, 0.8935999870300293]


## **Save and reuse the model**

In [ ]:
# get the best performing model
model = clf.export_model()
# display the architecture of the best performing model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None,)]                 0         
                                                                 
 expand_last_dim (ExpandLast  (None, 1)                0         
 Dim)                                                            
                                                                 
 text_vectorization (TextVec  (None, 512)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 512, 64)           320064    
                                                                 
 dropout (Dropout)           (None, 512, 64)           0         
                                                                 
 conv1d (Conv1D)             (None, 508, 256)          82176 

In [ ]:
from tensorflow.keras.models import load_model
# save the model on disk
model.save("text_classifier", save_format = "tf")
# load the model from disk
loaded_model = load_model("text_classifier",
                          custom_objects = ak.CUSTOM_OBJECTS)
# use loaded model for future predictions on new samples
loaded_model.predict(x_test)

INFO:tensorflow:Assets written to: text_classifier/assets


array([[1.4865190e-02],
       [1.5437603e-04],
       [1.9477427e-02],
       ...,
       [9.3894261e-01],
       [9.5961356e-01],
       [9.6271133e-01]], dtype=float32)

Write the result to disk

In [ ]:
# write the report to disk
		OUTPUT_PATH = "output"
        p = os.path.sep.join(OUTPUT_PATH, "{}.txt")
		f = open(p, "w")
		f.write(predicted_y)
		f.write("\nscore: {}".format(score))
		f.close()